In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, BitsAndBytesConfig

set_seed(1234)

prompt = "Will AI take over the world?"

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto", revision="float16")

%load_ext memory_profiler
import time

start_time = time.time()
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
%memit outputs = model.generate(**inputs, do_sample=True, max_new_tokens=150)
inference_time = time.time() - start_time
print("Inference time:", inference_time)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(result)

m_name = 'gemma-single-gpu-float16'
model.save_pretrained(m_name)
tokenizer.save_pretrained(m_name)
model.push_to_hub(m_name)
tokenizer.push_to_hub(m_name)

In [ ]:
!lm_eval --model hf --model_args pretrained=gemma-single-gpu-float16 --tasks winogrande,arc_challenge --device cuda:0 --num_fewshot 1 --batch_size 8 --output_path ./eval_harness/gemma-single-gpu-float16